In [ ]:
from xai import *
import torch

In [ ]:

try:
    dqn = DQN.load("dqn-model.pt")
except FileNotFoundError:
    print("Creating new agent...")
    dqn = DQN(autoencoder_path="asteroids-l32.pt",translate=True, rotate=True, device="cpu") 
    dqn.save("dqn-model.pt")

In [ ]:
dqn.train(
    total_time_steps=1_000_000,
    replay_buffer_size=GigaBytes(2),
    learning_rate = 1e-3,
    learning_starts = 50,
    batch_size = 16,
    tau = 1.0,
    gamma = 0.99,
    train_frequency = 32,
    frame_skip=4,
    gradient_steps = 2,
    episode_save_freq= 3,
    target_update_frequency = 2000,
    final_exploration_rate_progress = 0.03,
    initial_exploration_rate = 1.0,
    final_exploration_rate = 0.05,
    verbose = True
)

In [4]:
with Window("Asteroids", 60, 4.0) as window:
    for step in dqn.rollout(0.0, frame_skips=4).take(3000):
        window(step.observation.numpy(False))

WindowClosed: 